In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
data = model_data.copy()
# take one copy for model development data
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [5]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [6]:
features_list = ['Num_Enq_all_loans_last_9M',
'Num_of_PERSONAL LOAN_opened_last9M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_36M',
'num_CREDIT CARD_enq_last_18M',
'Total_outstanding_Balance_CREDIT CARD',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'num_PL_LE_1_Lac_opened_in_last_12M',
'num_PL_LE_1_Lac_with_no_delinquency',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_6M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'Utilization_all',
'num_of_loans_le_1lac_non_credit',
                 
# Reconcilated Features
'percentage_ever_delinquent',
'months_since_most_recent_delinquency_installment',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'num_debits_gt_10000_M3',
'num_credit_gt_salary',
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',
'cash_withdrawal_sum',
'num_cash_withdrawal_last_1_months',
'bounced_transactions_count',
'min_balance_last_3_months',
'days_balance_lt_5000_M2',
'days_balance_lt_10000_M3',
'avg_num_credits_last_2_months',
'Max_Credit_Avg_M1_M2_M3',
'ratio_3th_day_balance_to_max_credit_M2',
'ratio_active_loan_vs_salary',
'num_self_trans_last_3_months',
                 
'statedIncome',
'deviation_from_mean_income#last_30_Days',
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'yearOfBirth_mean',
'age',
'month_year',
'G15_B24']

In [7]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3046239/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3046239/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 40), (13738, 40), (20606,), (13738,))

In [10]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 40) (4122, 40) (13738, 40)


In [11]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [12]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.4,
    'colsample_bytree': 0.4,
    'lambda': 5,
    'alpha': 4.5,
    'gamma': 4.8,
    'random_state': 24
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 250,
    early_stopping_rounds= 50,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.52887	eval-auc:0.52411
[10]	train-auc:0.61612	eval-auc:0.61398
[20]	train-auc:0.62279	eval-auc:0.60998
[30]	train-auc:0.62313	eval-auc:0.61147
[40]	train-auc:0.62704	eval-auc:0.61597
[50]	train-auc:0.63030	eval-auc:0.61660
[60]	train-auc:0.63187	eval-auc:0.61691
[70]	train-auc:0.63077	eval-auc:0.61929
[80]	train-auc:0.63494	eval-auc:0.61460
[90]	train-auc:0.63565	eval-auc:0.61587
[100]	train-auc:0.63603	eval-auc:0.61757
[110]	train-auc:0.63639	eval-auc:0.61728
[120]	train-auc:0.63934	eval-auc:0.61862
[130]	train-auc:0.63902	eval-auc:0.61998
[140]	train-auc:0.64101	eval-auc:0.62204
[150]	train-auc:0.64293	eval-auc:0.62380
[160]	train-auc:0.64309	eval-auc:0.62587
[170]	train-auc:0.64449	eval-auc:0.62691
[180]	train-auc:0.64401	eval-auc:0.62725
[190]	train-auc:0.64474	eval-auc:0.62771
[200]	train-auc:0.64596	eval-auc:0.62922
[210]	train-auc:0.64632	eval-auc:0.62911
[220]	train-auc:0.64752	eval-auc:0.62916
[230]	train-auc:0.64815	eval-auc:0.62943
[240]	train-auc:0.64918	eva

In [13]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.65, Gini: 0.30, KS: 0.21
Eval AUC: 0.63, Gini: 0.26, KS: 0.20
Test AUC: 0.63, Gini: 0.26, KS: 0.18
OOT AUC: 0.63, Gini: 0.26, KS: 0.19


In [14]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.150394         0.116368        0.177364        0.101219   1649   
9          0.103155         0.094041        0.101208        0.087587   1648   
8          0.071602         0.081790        0.087584        0.076444   1648   
7          0.061249         0.071996        0.076442        0.068356   1649   
6          0.063714         0.066079        0.068355        0.064241   1648   
5          0.055218         0.062926        0.064239        0.061766   1648   
4          0.044876         0.060829        0.061766        0.059957   1649   
3          0.043083         0.059149        0.059956        0.058369   1648   
2          0.038228         0.057540        0.058368        0.056614   1648   
1          0.023044         0.055170        0.056612        0.052624   1649   

        label_good  label_bad  
Decile                         
10            1401        248  
9             1478        170  
8             1530        118  
7             1548        101  
6             1543        105  
5             1557         91  
4             1575         74  
3             1577         71  
2             1585         63  
1             1611         38

In [15]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.128093         0.116761        0.178900        0.101874   1374   
9          0.080786         0.094646        0.101873        0.087985   1374   
8          0.070597         0.082028        0.087981        0.076502   1374   
7          0.065550         0.072218        0.076490        0.068593   1373   
6          0.064047         0.066240        0.068590        0.064393   1374   
5          0.053130         0.063073        0.064391        0.061896   1374   
4          0.051712         0.060907        0.061895        0.060016   1373   
3          0.047307         0.059211        0.060012        0.058403   1374   
2          0.030568         0.057585        0.058402        0.056666   1374   
1          0.021106         0.055249        0.056666        0.052624   1374   

        label_good  label_bad  
Decile                         
10            1198        176  
9             1263        111  
8             1277         97  
7             1283         90  
6             1286         88  
5             1301         73  
4             1302         71  
3             1309         65  
2             1332         42  
1             1345         29

In [16]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.146161         0.118383        0.174092        0.102829   1081   
9          0.113889         0.094923        0.102824        0.088071   1080   
8          0.085185         0.081916        0.088068        0.076462   1080   
7          0.084259         0.072252        0.076450        0.068811   1080   
6          0.072222         0.066530        0.068806        0.064548   1080   
5          0.072222         0.063274        0.064541        0.062096   1080   
4          0.071296         0.061136        0.062094        0.060269   1080   
3          0.047222         0.059451        0.060269        0.058681   1080   
2          0.034259         0.057943        0.058679        0.057080   1080   
1          0.034228         0.055596        0.057079        0.052615   1081   

        label_good  label_bad  
Decile                         
10             923        158  
9              957        123  
8              988         92  
7              989         91  
6             1002         78  
5             1002         78  
4             1003         77  
3             1029         51  
2             1043         37  
1             1044         37

In [17]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
0                           Num_Enq_all_loans_last_9M        65.0
18                          Num_Enq_non_BANK_last_12M        57.0
26                         bounced_transactions_count        45.0
1                  Num_of_PERSONAL LOAN_opened_last9M        32.0
3                      num_CONSUMER LOAN_enq_last_36M        30.0
27                            days_balance_lt_5000_M2        24.0
12                                    Utilization_all        24.0
17                           No of enq in past 90days        21.0
7                          Age_of_oldest_open_account        21.0
11           Utilization_PERSONAL LOAN_opened_last_9M        18.0
5               Total_outstanding_Balance_CREDIT CARD        18.0
36                                                age        17.0
22                                    upi_trans_count        16.0
29                      avg_num_credits_last_2_months        16.0
21        

In [18]:
#Save the pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBLA_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_BBLA_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_BBLA_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_BBLA_Oot_Pred_1524.csv',index=False)


In [15]:
#Save
xgb_model.save_model('LeoPayu_BBLA_1_15_in_24mob.json')
print('Model Saved')

Model Saved
